In [46]:
from neo4j import GraphDatabase

In [47]:
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    # Queries
    def create_node(self, label, properties):
        with self.driver.session() as session:
            session.run(f"CREATE (n:{label} $properties", properties=properties)


    def degree_centrality(self):
        """
        Calculates the degree centrality for nodes in the graph, returning nodes with their centrality scores.

        Returns:
            list: A list of tuples containing (AgentId, score) for each node.

        Usage:
            Use this method to analyze the structure of the network and find key nodes
            that might be influential in information propagation or decision-making.
        """
        with self.driver.session() as session:
            result = session.run(f"""
            CALL gds.degree.stream('myGraph')
            YIELD nodeId, score
            WHERE gds.util.asNode(nodeId).AgentId IS NOT NULL
            RETURN gds.util.asNode(nodeId).AgentId AS AgentId, score
            ORDER BY score DESC
            """)

            # Convertir el resultado a una lista de diccionarios
            return [{"AgentId": record["AgentId"], "score": record["score"]} for record in result]


    def detect_client_communities(self):
        """
        Detects communities of clients in the graph using the label propagation algorithm,
        returning community identifiers and their members.

        Returns:
            list: A list of tuples containing (communityId, Members).

        Usage:
            Use this method to understand the structure of the client base or in social network
            analysis where it is useful to identify groups of users with common interests.
        """
        with self.driver.session() as session:
            result = session.run(f"""
            CALL gds.labelPropagation.stream('myGraph')
            YIELD nodeId, communityId
            RETURN communityId, collect(gds.util.asNode(nodeId).ClientId) AS Members
            ORDER BY size(Members) DESC
            """)

            # Convertir el resultado a una lista de diccionarios
            return [{"communityId": record["communityId"], "Members": record["Members"]} for record in result]


    def detect_agent_communities(self):
        """
        Detects communities of agents in the graph using the Louvain algorithm,
        returning community identifiers and their members.

        Returns:
            list: A list of tuples containing (communityId, Members).

        Usage:
            Use this method when you need to perform a deep analysis of the relationships
            among agents, especially in contexts where collaboration or interaction
            among them is relevant.
        """
        with self.driver.session() as session:
            result = session.run(f"""
            CALL gds.louvain.stream('myGraph')
            YIELD nodeId, communityId
            RETURN communityId, collect(gds.util.asNode(nodeId).AgentId) AS Members
            ORDER BY size(Members) DESC
            """)

            # Convertir el resultado a una lista de diccionarios
            return [{"communityId": record["communityId"], "Members": record["Members"]} for record in result]


    def node_similarity(self):
        """
        Calculates the similarity between nodes in the graph, returning pairs of nodes with their similarity scores.

        Returns:
            list: A list of tuples containing (Node1, Node2, similarity) for each pair.

        Usage:
            Use this method in recommendation systems or network analysis when you want
            to identify nodes that have similar characteristics or behaviors.
        """
        with self.driver.session() as session:
            result = session.run(f"""
            CALL gds.nodeSimilarity.stream('myGraph')
            YIELD node1, node2, similarity
            WHERE gds.util.asNode(node1).ClientId IS NOT NULL
            AND gds.util.asNode(node2).ClientId IS NOT NULL
            RETURN gds.util.asNode(node1).ClientId AS Node1, gds.util.asNode(node2).ClientId AS Node2, similarity
            ORDER BY similarity DESC
            LIMIT 10
            """)

            # Convertir el resultado a una lista de diccionarios
            return [{"Node1": record["Node1"], "Node2": record["Node2"], "similarity": record["similarity"]} for record in result]


    def get_nodes(self, label):
        with self.driver.session() as session:
            result = session.run(f"MATCH (n:{label}) RETURN n")
            return [record["n"] for record in result]


In [48]:
uri = "bolt://localhost:7687"  # Cambia esto si tu URI es diferente
user = "neo4j"                   # Nombre de usuario
password = "12345678"       # Contraseña

# Uso de la clase
neo4j_conn = Neo4jConnection(uri, user, password)

In [49]:
degree_centrality  = neo4j_conn.degree_centrality()
similarity_results = neo4j_conn.node_similarity()
community_results = neo4j_conn.detect_client_communities()
agent_community_results = neo4j_conn.detect_agent_communities()

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: "\n            CALL gds.labelPropagation.stream('myGraph')\n            YIELD nodeId, communityId\n            RETURN communityId, collect(gds.util.asNode(nodeId).ClientId) AS Members\n            ORDER BY size(Members) DESC\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: "\n            CALL gds.louvain.stream('myGraph')\n            YIELD nodeId, communityId\n            RETURN communityId, collect(gds.util.asNode(

In [50]:
degree_centrality

[{'AgentId': '3', 'score': 11.0},
 {'AgentId': '2', 'score': 10.0},
 {'AgentId': '7', 'score': 8.0},
 {'AgentId': '11', 'score': 8.0},
 {'AgentId': '1', 'score': 7.0},
 {'AgentId': '9', 'score': 7.0},
 {'AgentId': '10', 'score': 7.0},
 {'AgentId': '6', 'score': 6.0},
 {'AgentId': '12', 'score': 5.0},
 {'AgentId': '19', 'score': 5.0},
 {'AgentId': '5', 'score': 4.0},
 {'AgentId': '18', 'score': 4.0},
 {'AgentId': '4', 'score': 3.0},
 {'AgentId': '8', 'score': 3.0},
 {'AgentId': '14', 'score': 3.0},
 {'AgentId': '13', 'score': 2.0},
 {'AgentId': '15', 'score': 2.0},
 {'AgentId': '16', 'score': 2.0},
 {'AgentId': '20', 'score': 2.0},
 {'AgentId': '17', 'score': 1.0},
 {'AgentId': '1', 'score': 0.0},
 {'AgentId': '1', 'score': 0.0},
 {'AgentId': '3', 'score': 0.0},
 {'AgentId': '4', 'score': 0.0},
 {'AgentId': '3', 'score': 0.0},
 {'AgentId': '1', 'score': 0.0},
 {'AgentId': '2', 'score': 0.0},
 {'AgentId': '2', 'score': 0.0},
 {'AgentId': '9', 'score': 0.0},
 {'AgentId': '1', 'score': 0.0

In [51]:
similarity_results

[{'Node1': '6', 'Node2': '11', 'similarity': 0.3333333333333333},
 {'Node1': '4', 'Node2': '17', 'similarity': 0.3333333333333333},
 {'Node1': '17', 'Node2': '4', 'similarity': 0.3333333333333333},
 {'Node1': '11', 'Node2': '6', 'similarity': 0.3333333333333333},
 {'Node1': '9', 'Node2': '11', 'similarity': 0.25},
 {'Node1': '6', 'Node2': '9', 'similarity': 0.25},
 {'Node1': '6', 'Node2': '16', 'similarity': 0.25},
 {'Node1': '4', 'Node2': '20', 'similarity': 0.25},
 {'Node1': '6', 'Node2': '13', 'similarity': 0.25},
 {'Node1': '4', 'Node2': '14', 'similarity': 0.25}]

In [52]:
community_results

[{'communityId': 280, 'Members': ['1', '3', '6', '9', '11', '13', '16', '19']},
 {'communityId': 281, 'Members': ['2', '5', '8', '12', '15', '18']},
 {'communityId': 282, 'Members': ['4', '7', '10', '14', '17', '20']},
 {'communityId': 300, 'Members': ['1']},
 {'communityId': 306, 'Members': ['7']},
 {'communityId': 302, 'Members': ['3']},
 {'communityId': 303, 'Members': ['8']},
 {'communityId': 350, 'Members': ['25']},
 {'communityId': 328, 'Members': ['3']},
 {'communityId': 312, 'Members': ['13']},
 {'communityId': 321, 'Members': ['8']},
 {'communityId': 308, 'Members': ['3']},
 {'communityId': 323, 'Members': ['10']},
 {'communityId': 311, 'Members': ['12']},
 {'communityId': 318, 'Members': ['19']},
 {'communityId': 333, 'Members': ['8']},
 {'communityId': 310, 'Members': ['1']},
 {'communityId': 335, 'Members': ['10']},
 {'communityId': 315, 'Members': ['16']},
 {'communityId': 395, 'Members': ['70']},
 {'communityId': 317, 'Members': ['18']},
 {'communityId': 348, 'Members': [

In [53]:
agent_community_results

[{'communityId': 123,
  'Members': ['7',
   '10',
   '15',
   '16',
   '20',
   '3',
   '2',
   '7',
   '7',
   '7',
   '16',
   '12',
   '20',
   '7',
   '10',
   '7',
   '15',
   '10',
   '7',
   '11',
   '16',
   '2',
   '10',
   '7',
   '10',
   '10',
   '10',
   '9',
   '20',
   '15',
   '10']},
 {'communityId': 187,
  'Members': ['11',
   '18',
   '19',
   '1',
   '14',
   '11',
   '11',
   '18',
   '7',
   '11',
   '3',
   '8',
   '18',
   '19',
   '19',
   '19',
   '11',
   '19',
   '18',
   '11',
   '18',
   '19',
   '11']},
 {'communityId': 199,
  'Members': ['2',
   '5',
   '8',
   '2',
   '8',
   '3',
   '2',
   '5',
   '5',
   '2',
   '2',
   '3',
   '11',
   '2',
   '8',
   '5',
   '2',
   '5',
   '2',
   '9',
   '2']},
 {'communityId': 192,
  'Members': ['4',
   '12',
   '14',
   '4',
   '14',
   '12',
   '6',
   '4',
   '1',
   '4',
   '9',
   '12',
   '9',
   '12',
   '14',
   '12']},
 {'communityId': 190,
  'Members': ['6',
   '9',
   '13',
   '3',
   '9',
   '9',
   

In [54]:
agent_community_results

[{'communityId': 123,
  'Members': ['7',
   '10',
   '15',
   '16',
   '20',
   '3',
   '2',
   '7',
   '7',
   '7',
   '16',
   '12',
   '20',
   '7',
   '10',
   '7',
   '15',
   '10',
   '7',
   '11',
   '16',
   '2',
   '10',
   '7',
   '10',
   '10',
   '10',
   '9',
   '20',
   '15',
   '10']},
 {'communityId': 187,
  'Members': ['11',
   '18',
   '19',
   '1',
   '14',
   '11',
   '11',
   '18',
   '7',
   '11',
   '3',
   '8',
   '18',
   '19',
   '19',
   '19',
   '11',
   '19',
   '18',
   '11',
   '18',
   '19',
   '11']},
 {'communityId': 199,
  'Members': ['2',
   '5',
   '8',
   '2',
   '8',
   '3',
   '2',
   '5',
   '5',
   '2',
   '2',
   '3',
   '11',
   '2',
   '8',
   '5',
   '2',
   '5',
   '2',
   '9',
   '2']},
 {'communityId': 192,
  'Members': ['4',
   '12',
   '14',
   '4',
   '14',
   '12',
   '6',
   '4',
   '1',
   '4',
   '9',
   '12',
   '9',
   '12',
   '14',
   '12']},
 {'communityId': 190,
  'Members': ['6',
   '9',
   '13',
   '3',
   '9',
   '9',
   